# sparkle_motion — Colab / Drive setup
This notebook contains the recommended Colab setup steps and a small smoke-test harness for running the orchestrator in a GPU environment.
- Use the cells below to mount Google Drive (Colab only).
- Use the installation cell to install the ML stack from `requirements-ml.txt` (optional; heavy).
- The smoke-test cell is a safe stub that checks imports and shows how to run the orchestrator in simulation mode.

## Notes and expectations
- This notebook is intended for Google Colab (A100) runs. If you are running locally, skip the Drive mount and run the commands in a terminal.
- Before installing the heavy ML dependencies, ensure you have sufficient disk and GPU (Colab or a VM). The requirements are listed in `requirements-ml.txt`.

In [ ]:
# Cell 1: Mount Google Drive (Colab-only)
try:
    from google.colab import drive
    print('Mounting Google Drive at /content/drive...')
    drive.mount('/content/drive')
except Exception as e:
    print('Not running in Google Colab or google.colab is unavailable:', e)
    print('If you want to use Drive, mount it manually or run in Colab.')


In [ ]:
# Cell 2: Install ML dependencies from requirements-ml.txt (Colab / heavy)
import importlib.util
import subprocess


def sh(cmd):
    print('Running:', cmd)
    return subprocess.check_call(cmd, shell=True)


if importlib.util.find_spec('google.colab'):
    print('Detected Colab.\n')
    print('If you need a CUDA-optimized torch wheel, install it first as recommended in the repo notebook.\n')
    # The repository contains requirements-ml.txt at the repo root.
    # If you placed the repository under Drive, adjust the path accordingly (e.g. /content/drive/MyDrive/sparkle_motion/requirements-ml.txt).
    req_path = 'requirements-ml.txt'
    try:
        sh(f'pip install -r "{req_path}"')
    except subprocess.CalledProcessError as exc:
        print('pip install failed:', exc)
else:
    print('Not running in Colab — to install locally run:\n    pip install -r requirements-ml.txt')


In [ ]:
# Cell 3: Smoke-test stub for the orchestrator (safe, non-destructive)
# This cell attempts to import the orchestrator package and reports what is available.
try:
    import sparkle_motion.orchestrator as orchestrator_mod
    print('Imported sparkle_motion.orchestrator ->', orchestrator_mod)
    if hasattr(orchestrator_mod, 'Runner'):
        print('Runner class is available. You can instantiate it for a simulation run.')
        print('Example (local):')
        print("  from sparkle_motion.orchestrator import Runner")
        print("  r = Runner(run_dir='runs')")
        print("  # then use r.run(...) or similar per your orchestrator API")
    else:
        print('Runner class not found — inspect src/sparkle_motion/orchestrator.py for usage.')
except Exception as e:
    print('Could not import orchestrator module:', e)
    print('If you want to run the orchestrator, ensure the package is on PYTHONPATH (e.g., pip install -e .) or run via the repository root.')
